In [1]:
library(reticulate)
library(MASS)
library(tidyverse)
library(ggplot2)
library(GGally)
library(patchwork)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
use_condaenv("base")


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


Attaching package: ‘patchwork’


The following object is masked from ‘package:MASS’:

    area


Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:


In [2]:
setwd("/home/mnt/weka/nzh/team/woodsqu2/nzhanglab/project/linyx/footprints/github")
source("code/models.R")
source("code/utils.R")
source("code/visualizations.R")


### Load data, preprocess


##### Now we load cell barcodes with cell type information and mitochondria regions

In [3]:
barcodes = read.csv("data/mitochondria/bcanno.csv") ### the cells of interest with barcodes and grouping info
barcodes = barcodes[c('barcode', 'celltype')]
head(barcodes)


,barcode,celltype
,<chr>,<chr>
1,AAACGAAAGAGCACTG-1,PCT
2,AAACGAAAGGTTAACA-1,PCT
3,AAACGAAAGTTGACAA-1,PT_VCAM1
4,AAACGAACAATTGTGC-1,PST
5,AAACGAACACATGATC-1,PCT
6,AAACGAACAGGTAGCA-1,ICB


##### The cell type can be set at different resolution levels; for example, you can assign the same type to all cells if you want to create a sample pseudobulk

In [4]:
regionsBed = data.frame('chr' = 'chrM', 
                        'start' = seq(501, 16500, 1000), 
                        'end' = seq(1500, 16500, 1000)) ## mito regions
head(regionsBed, 3)


,chr,start,end
,<chr>,<dbl>,<dbl>
1,chrM,501,1500
2,chrM,1501,2500
3,chrM,2501,3500


##### Then we get base pair insertions from the mitochondria fragment file for each cell-type-specific pseudobulk, extracted from bam file. 

In [5]:
counts = get_count("data/mitochondria/fragments_collapse_within.tsv.gz", ### path to fragments
                   regionsBed, ### peak regions
                   barcodes) ### cells of interest
saveRDS(counts, 'data/mitochondria/counts.rds')


Make 1bp step .. 
Reformating counts data into a list (each element is data for a region) ..
[1] "Re-organizing data into lists"
[1] "2025-06-09 14:03:11.86856 Processing chunk 1 out of 1 chunks"
Generating matrix of counts for group BCELL..
Generating matrix of counts for group DCT1..
Generating matrix of counts for group DCT2_PC..
Generating matrix of counts for group FIB_VSMC_MC..
Generating matrix of counts for group ENDO..
Generating matrix of counts for group ICB..
Generating matrix of counts for group ICA..
Generating matrix of counts for group MONO..
Generating matrix of counts for group PEC..
Generating matrix of counts for group PST..
Generating matrix of counts for group PT_VCAM1..
Generating matrix of counts for group TCELL..
Generating matrix of counts for group PCT..
Generating matrix of counts for group PODO..
Generating matrix of counts for group PT_PROM1..
Generating matrix of counts for group TAL..


Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



  |======================================================================| 100%Done!
Time elapsed:  11.4461 secs  



##### Extract the DNA sequence within a local window (±50 bp) centered on each base pair


In [6]:
### base pair level
regions = data.frame('chr' = 'chrM', 
                        'start' = 501:16500, 
                        'end' = 501:16500)
ranges = IRanges::IRanges(start = regions$start, 
                          end = regions$end)
regions = makeGRangesFromDataFrame(regions)

contextRadius = 50
contextLen = 2*contextRadius + 1
regionSeqs = pbmcapply::pbmclapply(
  1:length(regions),
  function(regionInd){
    range = regions[regionInd]
    ### get the dna sequence for each base pair
    regionSeq = Biostrings::getSeq(BSgenome.Hsapiens.UCSC.hg38::BSgenome.Hsapiens.UCSC.hg38, ### reference genome
                                   as.character(seqnames(range)), 
                                   start = start(range) - contextRadius, ### window size 100
                                   width = width(range) + contextLen - 1,
                                   as.character = T)
    regionSeq
  },
  mc.cores = 8
)

context = do.call(rbind.data.frame, regionSeqs)


##### Identify base pairs whose local DNA sequence contains an 'N'.

In [7]:
index_n = which(apply(context, 1, function(x) grepl('N', x, fixed = TRUE)))


##### Create a DataFrame containing insertions, local window DNA sequences, and relative Tn5 insertion counts for each base pair in mitochondrial regions

In [8]:
insertions = NULL
for (i in 1:16){
  insertions = c(insertions, get_insertions(countData = counts, 
                                            regionInd = i, 
                                            groupIDs = unique(counts[[i]]$group), 
                                            width = 1000))
}

obsbias = data.frame('context' = context, 
                     'insertions' = as.numeric(insertions))
obsbias$positions = 501:16500
colnames(obsbias) = c('context', 'insertions', 'positions')

obsbias$insertions = as.numeric(obsbias$insertions)
obsbias$positions = as.numeric(obsbias$positions)
obsbias['obs_bias'] = apply(obsbias, 1, function(x){
  as.numeric(x[2])/as.numeric(mean(obsbias[as.numeric(obsbias$positions) >= as.numeric(x[3]) - 50 & 
                                             as.numeric(obsbias$positions) <= as.numeric(x[3]) + 50, "insertions"]))
  }
)

rownames(obsbias) = NULL
obsbias$BACInd = rep(1:16, each = 1000)


In [9]:
head(obsbias)


,context,insertions,positions,obs_bias,BACInd
,<chr>,<dbl>,<dbl>,<dbl>,<int>
1,ATTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAA,797,501,0.5085388,1
2,TTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAAC,3029,502,1.9687762,1
3,TTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACC,264,503,0.1731726,1
4,TTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCA,303,504,0.1983297,1
5,TCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAA,1093,505,0.7164565,1
6,CCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAAA,942,506,0.6271190,1


##### Save the observed bias in mitochondria. Since the input only accepts ATCG, exclude base pairs whose local context contains 'N'

In [10]:
write.table(obsbias, 'data/footprints_identification/obsBias.tsv', sep = "\t")
obsBias_finetuned = obsbias[-index_n, ]
write.table(obsBias_finetuned, 'data/footprints_identification/obsBias_finetuned.tsv', sep = "\t")


### Finetuned the model with observed bias in mitochondria. 


In [ ]:
finetuned_model(code_path = "code/", 
                obsbias_path = "data/mitochondria/obsBias_finetuned.tsv", 
                PRINT_model_path = "data/shared/Tn5_NN_model.h5", 
                finetuned_model_save_path = "data/shared/", 
                finetuned_model_name = "Tn5_NN_model_Control_6_freezed_finetuned.h5"
                )


### Mitochondrial in-sample fdr controls

#### Load insertions and predicted bias derived from finetuning

In [ ]:
mito_counts = readRDS('data/mitochondria/counts.rds')
mito_regions = data.frame('chr' = 'chrM', 
                        'start' = seq(501, 16500, 1000), 
                        'end' = seq(1500, 16500, 1000))


In [ ]:
get_bias(regionsBed, ### peak file
         referenceGenome = 'hg38',  ### reference genome
         path = paste0(getwd(), '/data/mitochondria/freezed_finetuned_'), ### save path
         model_path = 'data/shared/', ### path to the Tn5 model
         code_path = 'code/', ### path to code
         model_use = 'Tn5_NN_model_Control_6_freezed_finetuned.h5') ### name of Tn5 model


In [ ]:
mito_bias = read.table("data/mitochondria/freezed_finetuned_pred_bias.txt")


In [ ]:
### takes long, better run in parallel. 
if (F) {
    logp_threshold = mito_fdr(mito_counts = mito_counts, 
                              mito_bias = mito_bias, 
                              mito_regions = mito_regions, 
                              i = 11, ### test region
                              seeds = 1:100, ### downsampling seed, repeat for 100 times 
                              average_insertions = c(seq(0.1, 5, 0.1), seq(5.5, 40, 0.5)), ### downsample target: average insertions
                              alpha = 0.05) ### fdr 
    write.table(logp_threshold, 'data/mitochondria/logp_threshold.txt')
}


In [ ]:
logp_threshold = read.table("data/mitochondria/logp_threshold.txt")
head(logp_threshold) 
